### Modules

In [3]:
import pandas as pd
import logging
from youtube_search import YoutubeSearch
from pytubefix import YouTube
from pydub import AudioSegment

import re
import os
import datetime

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

os.makedirs('datasets/songs', exist_ok=True)
os.makedirs('datasets/wav_songs', exist_ok=True)

### Variables
```yaml
This module contains the configuration for the youtube search and download
You can change the configuration here

YOUTUBE_BASE_URL: The base url for the youtube search
YOUTUBE_SEARCH_MAX_RESULTS: The maximum number of results to return from the youtube search
YOUTUBE_CLIENT_TYPE: The type of client to use for the youtube search
YOUTUBE_MAX_VIDEO_LENGTH: The maximum length of a video in seconds
YOUTUBE_MIN_VIDEO_LENGTH: The minimum length of a video in seconds

SONG_SIZE_PER_TITLE: size (count) songs per title
SEGMENT_LENGTH: The length of each segment in seconds

LAGU_DAERAH_PATH: The path to the lagu daerah dataset
DOWNLOADED_SONGS_PATH: The path to the downloaded songs
SONGS_WAV_PATH: The path to the songs in wav format
```

In [4]:

YOUTUBE_BASE_URL = 'https://www.youtube.com/watch?v='
YOUTUBE_SEARCH_MAX_RESULTS = 15
YOUTUBE_CLIENT_TYPE='IOS'
YOUTUBE_MAX_VIDEO_LENGTH = 500
YOUTUBE_MIN_VIDEO_LENGTH = 30

SONG_SIZE_PER_TITLE=10
SEGMENT_LENGTH = 30

# LAGU_DAERAH_PATH = 'data/lagu_daerah.csv'
LAGU_DAERAH_PATH = 'data/lagu_daerah_alternatif.csv'
DOWNLOADED_SONGS_PATH = 'data/downloaded_songs.csv'
SONGS_WAV_PATH = 'data/songs_wav.csv'

### Load Scrapping Result

In [5]:
data = pd.read_csv(LAGU_DAERAH_PATH, sep=',')
data.head()

,nama_lagu,asal
0,Bungong Jeumpa,Aceh
1,Mecepat – Cepetan,Bali
2,Sarinande,Bali
3,Putri,Bali
4,Ratu Anom,Bali


### Search and Download Music Function

In [6]:
def search_yt(query, max_results=YOUTUBE_SEARCH_MAX_RESULTS):
    try:
        results = YoutubeSearch(query, max_results).to_dict()
        for result in results:
            result['url'] = YOUTUBE_BASE_URL + result['id']
        return results
    except Exception as e:
        logging.error(f"Error searching YouTube for query '{query}': {e}")
        return []

def normalized_yt_title(title):
    text = title.lower()
    text = text.replace(' ', '_')
    text = re.sub(r'[^a-z0-9_]', '', text)
    text = re.sub(r'_{2,}', '_', text)
    
    return text

def dl_video(url):
    try:
        yt = YouTube(url, client=YOUTUBE_CLIENT_TYPE)
        logging.info(f'Downloading {yt.title}...')
        
        normalized_title = normalized_yt_title(yt.title)
        
        # check if video already downloaded
        filepath= f'datasets/songs/{normalized_title}.mp3'
        if os.path.exists(filepath):
            logging.info(f'File already exists: {filepath}')
            return filepath
        
        audio_stream = yt.streams.get_audio_only()
        
        audio_stream.download(output_path='datasets/songs', filename=normalized_title, mp3=True)
        
        return f'datasets/songs/{normalized_title}.mp3'
    except Exception as e:
        logging.error(f"Error downloading video from URL '{url}': {e}")
        return None
    
def parse_duration(duration_str):
    duration_str = duration_str.replace('.', ':')
    if duration_str.count(':') == 2:
        duration = datetime.datetime.strptime(duration_str, '%H:%M:%S')
    else:
        duration = datetime.datetime.strptime(duration_str, '%M:%S')
        
    return duration.hour * 3600 + duration.minute * 60 + duration.second

In [9]:
dl_results = []

In [10]:
for index, row in data[:30].iterrows():
    # Construct the search keyword
    keyword = f"Lagu Daerah {row['nama_lagu']} asal {row['asal']}"
    searched_songs = search_yt(keyword)
    
    downloaded_count = 0
    for song in searched_songs:
        if downloaded_count >= SONG_SIZE_PER_TITLE:
            logging.info(f"Downloaded {downloaded_count} songs for '{keyword}'")
            break
        
        try:
            duration = parse_duration(song['duration'])
            if duration < YOUTUBE_MAX_VIDEO_LENGTH and duration > YOUTUBE_MIN_VIDEO_LENGTH:
                path = dl_video(song['url'])
                
                # Append the result to downloaded_results list
                dl_results.append({
                    'title': song['title'],
                    'nama_lagu': row['nama_lagu'],
                    'region': row['asal'],
                    'keyword': keyword,
                    'duration': duration,
                    'url': song['url'],
                    'path': path
                })
                logging.info(f"Downloaded: {song['title']} from {song['url']}")
                
                downloaded_count += 1
            else:
                logging.warning(f"Duration of {song['title']} is too long: {duration} seconds")
        except Exception as e:
            logging.error(f"Error processing song {song['title']}: {e}")

2024-11-28 17:49:04,257 - INFO - Downloading Bungong Jeumpa | Lirik dan Terjemahan | Lagu Daerah Aceh | Dongeng Kita...
2024-11-28 17:49:04,258 - INFO - File already exists: datasets/songs/bungong_jeumpa_lirik_dan_terjemahan_lagu_daerah_aceh_dongeng_kita.mp3
2024-11-28 17:49:04,258 - INFO - Downloaded: Bungong Jeumpa | Lirik dan Terjemahan | Lagu Daerah Aceh | Dongeng Kita from https://www.youtube.com/watch?v=lAVi2OE2bRY
2024-11-28 17:49:04,767 - INFO - Downloading VIDEO LIRIK- BUNGONG JEUMPA...
2024-11-28 17:49:04,771 - INFO - File already exists: datasets/songs/video_lirik_bungong_jeumpa.mp3
2024-11-28 17:49:04,778 - INFO - Downloaded: VIDEO LIRIK- BUNGONG JEUMPA from https://www.youtube.com/watch?v=tgc0dBKcqhE
2024-11-28 17:49:04,996 - INFO - Downloading Bungong Jeumpa - Putri Ariani Cover (Lagu Daerah Aceh)...
2024-11-28 17:49:04,997 - INFO - File already exists: datasets/songs/bungong_jeumpa_putri_ariani_cover_lagu_daerah_aceh.mp3
2024-11-28 17:49:04,998 - INFO - Downloaded: Bungo

### Save list of songs to csv

In [11]:
df = pd.DataFrame(dl_results)

df.to_csv('data/downloaded_songs.csv', index=False)

### Convert to wav format

In [12]:
def convert_to_wav(path):
    if path is None:
        logging.error('File path is None')
        return None
    
    try:
        wav_path = path.replace('songs', 'wav_songs').replace('.mp3', '.wav')
        
        if os.path.exists(wav_path):
            logging.info(f'File already exists: {wav_path}')
            return wav_path
        
        logging.info(f'Converting {path} to {wav_path}')
        
        audio = AudioSegment.from_file(path)
        audio.export(wav_path, format='wav')
        
        return wav_path
    except Exception as e:
        logging.error(f'Exception occurred: {e}')
        return None

In [13]:
df['wav_path'] = df['path'].apply(convert_to_wav)

2024-11-28 18:10:35,204 - INFO - File already exists: datasets/wav_songs/bungong_jeumpa_lirik_dan_terjemahan_lagu_daerah_aceh_dongeng_kita.wav
2024-11-28 18:10:35,206 - INFO - Converting datasets/songs/video_lirik_bungong_jeumpa.mp3 to datasets/wav_songs/video_lirik_bungong_jeumpa.wav
2024-11-28 18:10:36,454 - INFO - File already exists: datasets/wav_songs/bungong_jeumpa_putri_ariani_cover_lagu_daerah_aceh.wav
2024-11-28 18:10:36,455 - INFO - Converting datasets/songs/bungong_jeumpa.mp3 to datasets/wav_songs/bungong_jeumpa.wav
2024-11-28 18:10:37,407 - INFO - Converting datasets/songs/bungong_jeumpa_tania.mp3 to datasets/wav_songs/bungong_jeumpa_tania.wav
2024-11-28 18:10:38,337 - INFO - File already exists: datasets/wav_songs/tari_bungong_jeumpa_aceh_rumah_belajar_mawinsya.wav
2024-11-28 18:10:38,338 - INFO - File already exists: datasets/wav_songs/kaka_alfarisi_bungong_jeumpa_official_video.wav
2024-11-28 18:10:38,339 - INFO - Converting datasets/songs/bungong_jeumpa_lagu_daerah_aceh

In [ ]:
df = df[df['wav_path'].notnull()]

In [16]:
df

,title,nama_lagu,region,keyword,duration,url,path,wav_path
0,Bungong Jeumpa | Lirik dan Terjemahan | Lagu D...,Bungong Jeumpa,Aceh,Lagu Daerah Bungong Jeumpa asal Aceh,243,https://www.youtube.com/watch?v=lAVi2OE2bRY,datasets/songs/bungong_jeumpa_lirik_dan_terjem...,datasets/wav_songs/bungong_jeumpa_lirik_dan_te...
1,VIDEO LIRIK- BUNGONG JEUMPA,Bungong Jeumpa,Aceh,Lagu Daerah Bungong Jeumpa asal Aceh,264,https://www.youtube.com/watch?v=tgc0dBKcqhE,datasets/songs/video_lirik_bungong_jeumpa.mp3,datasets/wav_songs/video_lirik_bungong_jeumpa.wav
2,Bungong Jeumpa - Putri Ariani Cover (Lagu Daer...,Bungong Jeumpa,Aceh,Lagu Daerah Bungong Jeumpa asal Aceh,160,https://www.youtube.com/watch?v=RjI7W_XmzLM,datasets/songs/bungong_jeumpa_putri_ariani_cov...,datasets/wav_songs/bungong_jeumpa_putri_ariani...
3,Bungong Jeumpa,Bungong Jeumpa,Aceh,Lagu Daerah Bungong Jeumpa asal Aceh,259,https://www.youtube.com/watch?v=_M1ZNoY99nk,datasets/songs/bungong_jeumpa.mp3,datasets/wav_songs/bungong_jeumpa.wav
4,Bungong Jeumpa - Tania,Bungong Jeumpa,Aceh,Lagu Daerah Bungong Jeumpa asal Aceh,218,https://www.youtube.com/watch?v=ZfZxIR9vRmk,datasets/songs/bungong_jeumpa_tania.mp3,datasets/wav_songs/bungong_jeumpa_tania.wav
...,...,...,...,...,...,...,...,...
295,lagu daerah gundul gundul pacul versi karaoke ...,Gundul Pacul,Jawa Tengah,Lagu Daerah Gundul Pacul asal Jawa Tengah,75,https://www.youtube.com/watch?v=YbagheN_quo,datasets/songs/lagu_daerah_gundul_gundul_pacul...,datasets/wav_songs/lagu_daerah_gundul_gundul_p...
296,GUNDUL GUNDUL PACUL - Javanese Gamelan Music J...,Gundul Pacul,Jawa Tengah,Lagu Daerah Gundul Pacul asal Jawa Tengah,251,https://www.youtube.com/watch?v=GKIulLNyXH4,datasets/songs/gundul_gundul_pacul_javanese_ga...,datasets/wav_songs/gundul_gundul_pacul_javanes...
297,"GUNDUL GUNDUL PACUL, SUWE ORA JAMU, CUBLAK CUB...",Gundul Pacul,Jawa Tengah,Lagu Daerah Gundul Pacul asal Jawa Tengah,345,https://www.youtube.com/watch?v=zvMqemh2sbg,datasets/songs/gundul_gundul_pacul_suwe_ora_ja...,datasets/wav_songs/gundul_gundul_pacul_suwe_or...
298,Video Lirik Lagu Hits | Gundul-Gundul Pacul,Gundul Pacul,Jawa Tengah,Lagu Daerah Gundul Pacul asal Jawa Tengah,137,https://www.youtube.com/watch?v=bAcB_R5gDMI,datasets/songs/video_lirik_lagu_hits_gundulgun...,datasets/wav_songs/video_lirik_lagu_hits_gundu...


In [17]:
df.to_csv('data/songs_wav.csv', index=False)